# FMLIKH Model: Small Amplitude Oscillatory Shear (SAOS)

## Protocol-Specific Context (Multi-Mode)

**FMLIKH SAOS** produces **broadened relaxation spectra** spanning decades of frequency:

1. **Multiple crossover frequencies**: $\omega_c,i \sim 1/\tau_i$ for each mode
2. **Composite Cole-Cole**: Superposition of depressed arcs with angle(s) $\theta_i = (1-\alpha_i)\pi/2$
3. **Extended power-law regimes**: $G' \sim \omega^{\alpha}$ over wider frequency range

**Why this matters**: Single-mode FIKH has limited frequency range. FMLIKH with N=3 can capture **broad experimental spectra** from 0.001 Hz to 100 Hz — fast modes respond at high frequency, slow modes at low frequency.

> **Physical insight**: Each structural level contributes to viscoelasticity at its characteristic frequency $\omega \sim 1/\tau_i$. Fast aggregates respond quickly (high $\omega$), slow networks respond slowly (low $\omega$). Fractional kinetics broadens each mode's contribution.

> **Handbook:** See [FMLIKH SAOS](../../docs/source/models/fikh/fmlikh.rst) for multi-mode Cole-Cole analysis.

## Learning Objectives

1. Compute FMLIKH SAOS moduli with N=3 modes
2. Observe broadened frequency response (vs single-mode)
3. Analyze mode contributions to $G'$, $G''$ at different frequencies
4. Plot composite Cole-Cole and identify mode features
5. Extract relaxation spectrum from SAOS data

## Prerequisites

- NB07: FMLIKH flow curve
- NB05: FIKH SAOS (single-mode baseline)

**Estimated Time:** 4-6 minutes (fast), 15-20 minutes (full)

## 1. Setup

In [ ]:
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"

In [ ]:
%matplotlib inline
import os
import sys
import time
import warnings

# Robust path resolution for execution from any directory
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fikh import FMLIKH

_nb_dir = Path(__file__).parent if "__file__" in dir() else Path.cwd()
_utils_candidates = [_nb_dir / ".." / "utils", Path("examples/utils"), _nb_dir.parent / "utils"]
for _p in _utils_candidates:
    if (_p / "fikh_tutorial_utils.py").exists():
        sys.path.insert(0, str(_p.resolve()))
        break
from fikh_tutorial_utils import (
    generate_synthetic_saos,
    get_fmlikh_param_names,
    load_fikh_parameters,
    print_convergence_summary,
    save_fikh_results,
    set_model_parameters,
)

# Shared plotting utilities
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

jax, jnp = safe_import_jax()
verify_float64()

# ============================================================
# FAST_MODE Configuration
# ============================================================
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

if FAST_MODE:
    print("FAST_MODE: reduced data/samples for quick validation")
    N_TIME_POINTS = 50  # Reduced from 200
    NUM_WARMUP = 50
    NUM_SAMPLES = 100
    NUM_CHAINS = 1
else:
    print("FULL mode: complete Bayesian inference")
    N_TIME_POINTS = 200  # Full resolution
    NUM_WARMUP = 200
    NUM_SAMPLES = 500
    NUM_CHAINS = 1

## 2. Load Calibrated Parameters

In [ ]:
N_MODES = 3
model = FMLIKH(n_modes=N_MODES, include_thermal=False, shared_alpha=True, alpha_structure=0.7)

try:
    params = load_fikh_parameters("fmlikh", "flow_curve")
    set_model_parameters(model, params)
    print("Loaded parameters from NB07")
except FileNotFoundError:
    print("Using default parameters")

## 3. Generate Synthetic Data

In [ ]:
OMEGA_RANGE = (0.01, 100.0)
omega_data, G_prime_data, G_double_prime_data = generate_synthetic_saos(
    model, omega_range=OMEGA_RANGE, n_points=40, noise_level=0.03, seed=42
)

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(omega_data, G_prime_data, "ko", markersize=7, label="G'")
ax.loglog(omega_data, G_double_prime_data, "s", color="gray", markersize=6, label="G''")
ax.set_xlabel("Frequency [rad/s]")
ax.set_ylabel("Modulus [Pa]")
ax.set_title("Synthetic FMLIKH SAOS Data")
ax.legend()
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## 4. Mode Contribution Analysis

In [ ]:
# Each mode has characteristic frequency 1/tau_i
mode_info = model.get_mode_info()
omega_fine = np.logspace(np.log10(OMEGA_RANGE[0]), np.log10(OMEGA_RANGE[1]), 100)

print("Mode characteristic frequencies:")
for m in mode_info["modes"]:
    omega_c = 1.0 / m["tau"] if m["tau"] > 0 else float("inf")
    print(f"  Mode {m['mode']}: ω_c = {omega_c:.4g} rad/s (τ = {m['tau']:.4g} s)")

## 5. Bayesian Inference

In [ ]:
model_fit = FMLIKH(n_modes=N_MODES, include_thermal=False, shared_alpha=True, alpha_structure=0.5)
G_star_data = G_prime_data + 1j * G_double_prime_data

# For SAOS, create synthetic time-domain oscillation data
# Use low frequency for fitting
omega_fit = np.median(omega_data)
period = 2 * np.pi / omega_fit
t_fit = np.linspace(0, 5 * period, N_TIME_POINTS)
gamma_0 = 0.01
strain_fit = gamma_0 * np.sin(omega_fit * t_fit)

# Use G_prime as proxy for stress magnitude (simplified)
stress_fit = G_prime_data[len(G_prime_data)//2] * strain_fit

model_fit.fit(t_fit, stress_fit, test_mode="startup", strain=strain_fit, method='scipy')

param_names = get_fmlikh_param_names(n_modes=N_MODES, shared_alpha=True)
initial_values = {n: model_fit.parameters.get_value(n) for n in param_names if n in model_fit.parameters}

print(f"Running NUTS: {NUM_WARMUP} warmup + {NUM_SAMPLES} samples x {NUM_CHAINS} chain(s)")
print(f"  FAST_MODE={FAST_MODE}, time points={len(t_fit)}")

t0 = time.time()
result = model_fit.fit_bayesian(
    t_fit, stress_fit, test_mode="startup", strain=strain_fit,
    num_warmup=NUM_WARMUP, num_samples=NUM_SAMPLES, num_chains=NUM_CHAINS,
    initial_values=initial_values, seed=42,
)
print(f"Bayesian time: {time.time() - t0:.1f} s")

In [ ]:
key_params = ["G_0", "G_1", "alpha_structure"]
print_convergence_summary(result, key_params)

print("\n### Diagnostic Targets")
print("| Metric | Target | Interpretation |")
print("|--------|--------|----------------|")
print("| **R-hat** | < 1.01 | Convergence |")
print("| **ESS** | > 400 | Sample count |")
print("| **Divergences** | < 1% | Quality |")

display_arviz_diagnostics(result, key_params, fast_mode=FAST_MODE)

### Convergence Diagnostics

**Diagnostic Targets:**

| Metric | Target | Interpretation |
|--------|--------|----------------|
| **R-hat** | < 1.01 | Convergence |
| **ESS** | > 400 | Sample count |
| **Divergences** | < 1% | Quality |

## 6. Save Results

In [ ]:
save_fikh_results(model_fit, result, "fmlikh", "saos", param_names)

## Key Takeaways

1. **FMLIKH SAOS captures broad frequency spectra** (vs single-mode limits)
2. **Multiple crossover frequencies** $\omega_c,i \sim 1/\tau_i$ from mode distribution
3. **Composite Cole-Cole** superposition of depressed arcs
4. **Extended power-law regimes** over wider $\omega$ range
5. **Mode decomposition** shows structural level contributions vs frequency
6. **Residual analysis** of moduli validates multi-mode relaxation spectrum

---

## Further Reading

- **[FMLIKH SAOS Analysis](../../docs/source/models/fikh/fmlikh.rst#stretched-exponential-vs-fractional-modes)**: Frequency-domain mode superposition
- **[Cole-Cole Decomposition](../../docs/source/models/fikh/fmlikh.rst#shared-vs-per-mode-fractional-order)**: Identifying mode features in complex plane

### Key References

1. Jaishankar, A. & McKinley, G.H. (2014). "A fractional K-BKZ constitutive formulation." *J. Rheol.*, 58, 1751-1788.
2. Wei, Y. et al. (2018). "A multimode structural kinetics constitutive equation." *J. Rheol.*, 62(1), 321-342.

### Next Steps

**Next**: NB12 (LAOS) — multi-mode harmonics and nonlinear oscillatory response